In [1]:
import json
import pickle
import numpy
import pandas
from pandas import DataFrame
from dtw import accelerated_dtw
import math

#pandas.set_option('display.max_rows', 100)

epLat = 0.012160099999998054
epLng = 0.014610749999992125
epTime = 255

with open('trajTrain.pickle', 'rb') as f:
  trajTrainDict = pickle.load(f)
  trajTrain = trajTrainDict['trajTrain']
  trajTrainOri = trajTrainDict['trajTrainOri']
  mappingTrain = trajTrainDict['mappingTrain']

with open('trajTest.pickle', 'rb') as f:
    trajTrainDict = pickle.load(f)
    trajTest = trajTrainDict['trajTest']
    trajTestOri = trajTrainDict['trajTestOri']
    mappingTest = trajTrainDict['mappingTest']

In [8]:
def ignoreBeginOrLastOnePointCube_Origi(traj, trajOri, mapping):
  hist = numpy.bincount(mapping[4])
  hist2 = numpy.bincount(hist)
  ratioOfMostGap = 0.2
  lessGap = numpy.where((hist2.cumsum() / hist2.sum()) <= ratioOfMostGap)[0][-1]
  # get lessGap
  n = hist.__len__()
  indexTop = 0
  indexTop2 = 0
  for i in range(n):
    if hist[i] <= lessGap:
      print('ignoreBeginOrLastOnePointCube head found')
      indexTop += 1
      indexTop2 += 1
    else:
      break
  indexEnd = traj.shape[0] - 1
  indexEnd2 = trajOri.shape[0] - 1
  for i in range(n - 1, 0, -1):
    if hist[i] <= lessGap:
      print('ignoreBeginOrLastOnePointCube tail found')
      indexEnd -= 1
      indexEnd2 -= 1
    else:
      break
  return (traj.iloc[indexTop:(indexEnd + 1), :].copy(), trajOri.iloc[indexTop2:(indexEnd2 + 1), :].copy())


In [2]:
def ignoreBeginOrLastOnePointCube(traj, trajOri, mapping, epLng, epLat):
    trajAmount = traj.shape[0]
    #print('trajAmount=',trajAmount)
    indexTop = 0
    while ( abs(traj.iloc[indexTop,3] - traj.iloc[indexTop+1,3]) < (epLng*3/4) ) & \
            (abs(traj.iloc[indexTop,4] - traj.iloc[indexTop+1,4]) < (epLat*3/4) ):
        indexTop += 1
        if indexTop >= (trajAmount - 2):
            return None
            #indexTop == (trajAmount-1) 表超過最後一個的index。最後一個無條件刪，故再-1
    indexTop2 = mapping.loc[ mapping[4] == indexTop,3].min()
    indexEnd2 = mapping.loc[ mapping[4] == traj.iloc[-2, 1], 3].max()
    return (traj.iloc[indexTop:-1, :].copy(), trajOri.iloc[indexTop2:(indexEnd2 + 1), :].copy() )
#第一cube啟始時間是最上時，最後cube是最下時(最早)

In [2]:
mappingTest

,0,1,2,3,4
0,0,0,1,0,0
1,0,0,1,1,0
2,0,0,1,2,0
3,0,0,1,3,0
4,0,0,1,4,0
...,...,...,...,...,...
47462,0,0,1400,15,2
47463,0,0,1400,16,3
47464,0,0,1400,17,3
47465,0,0,1400,18,4


In [9]:



def getOverlayOneCubeWithPointAmount(trajList1, trajList2, dtime, distX, distY, angDiffConst, pointAmountRatio=0):
    maxTime = trajList1[9] + dtime
    minTime = trajList1[9] - dtime
    trajList3 = trajList2.loc[(trajList2[9] > minTime) & (trajList2[9] < maxTime),]
    trajList4 = trajList3.loc[abs(trajList3[9] - trajList1[9]) < dtime,]
    trajList4 = trajList4.loc[abs(trajList4[3] - trajList1[3]) < distX,]
    trajList4 = trajList4.loc[abs(trajList4[4] - trajList1[4]) < distY,]
    trajList4 = trajList4.loc[angleDiff(trajList4[8], trajList1[8]) < angDiffConst,]
    while pointAmountRatio > 0:
      trajList5 = trajList4.loc[abs(trajList4[0] - trajList1[0]) < (trajList1[0] * (1 - pointAmountRatio)),]
      if trajList5.size > 0:
        trajList4 = trajList5
        break
      pointAmountRatio -= 0.1
    if trajList4.size <= 0:
      return None
    n1 = trajList4.shape[0]
    area1 = DataFrame(numpy.zeros((n1, 3), dtype=int))
    area1[0] = trajList4[5].values  # 其它軌
    area1[1] = trajList4[1].values  # 其它軌第幾cube
    area1[2] = trajList4.index.values  # 其它軌此cube在總名單上第幾
    return area1

def getOverlayOneCube(trajList1,trajList2,dtime,distX,distY,angDiffConst):
	maxTime = trajList1[9] + dtime
	minTime = trajList1[9] - dtime
	trajList3 = trajList2.loc[(trajList2[9] > minTime) & (trajList2[9] < maxTime),]
	trajList4 = trajList3.loc[abs(trajList3[9] - trajList1[9]) < dtime,]
	trajList4 = trajList4.loc[abs(trajList4[3] - trajList1[3]) < distX,]
	trajList4 = trajList4.loc[abs(trajList4[4] - trajList1[4]) < distY,]
	trajList4 = trajList4.loc[angleDiff(trajList4[8],trajList1[8]) < angDiffConst,]
	n1 = trajList4.shape[0]
	area1 = DataFrame(numpy.zeros((n1,3), dtype=int))
	area1[0] = trajList4[5].values  #其它軌
	area1[1] = trajList4[1].values  #其它軌第幾cube
	area1[2] = trajList4.index.values  #其它軌此cube在總名單上第幾
	return area1

def angleDiff(a ,b):
	result = abs(a - b)
	result = result.values
	if len(a) > 1:
		index = result > 180
		result[index] = 360 - result[index]
	else:
		if result > 180:
			result = 360 - result
	return result



from math import cos
from math import sin
from math import radians
from math import asin
import numpy

def getDistance(x,y):
	Long1,Lat1 = x
	Long2,Lat2 = y
	deltaX = cos(radians(Lat2))*cos(radians(Long2)) \
			 - cos(radians(Lat1)) * cos(radians(Long1))
	deltaY = cos(radians(Lat2))*sin(radians(Long2)) \
			 - cos(radians(Lat1)) * sin(radians(Long1))
	deltaZ = sin(radians(Lat2)) - sin(radians(Lat1))
	C = numpy.linalg.norm(numpy.array([deltaX, deltaY, deltaZ]))
	deltaSigma = 2 * asin(C/2)
	d = round( 6371 * 1000 * deltaSigma )
	return d


def getDTWeachRow(row,x,mappingTrain,trajTrainOri):
    tempTrajOriNo = mappingTrain.loc[
            (mappingTrain[2] == row[5]) &
            (mappingTrain[4] == row[1])
            ,3].to_list()
    y = trajTrainOri.loc[
            (trajTrainOri[5] == row[5]) &
            (trajTrainOri[1].isin( tempTrajOriNo ))
            ,[3,4]].to_numpy()
    d, cost_matrix, acc_cost_matrix, path = accelerated_dtw(x,y,getDistance)
    return d


def getTimeByMinDtw(targetNo,
                    area,
                    targetTrajOri,
                    targetMapping,
                    trajTrain,
                    trajTrainOri,
                    mappingTrain):
  trajTrainCubesIndex = area[2]

  targetTrajOriResetIndex = targetTrajOri.set_index(1)
  x = targetTrajOriResetIndex.loc[
    targetMapping.loc[targetMapping[4] == targetNo, 3],
    [3, 4]
  ].to_numpy()

  # result = []
  dfTrajNoWiCubeNo = trajTrain.loc[trajTrainCubesIndex, [5, 1]]
  n = dfTrajNoWiCubeNo.shape[0]
  result = dfTrajNoWiCubeNo.apply(getDTWeachRow, axis=1, args=(x, mappingTrain, trajTrainOri))
  minIndex = result.idxmin()
  minTrajTrain = trajTrain.loc[minIndex,]
  commonTime = trajTrain.loc[minIndex, 10] - trajTrain.loc[minIndex, 9]
  return (commonTime,minIndex,result[minIndex])

def getTrainNext(trajTrain,minIndex):
  nextIndex = minIndex+1
  preIndex = minIndex-1

  if (nextIndex in trajTrain.index) & \
     (trajTrain.loc[minIndex,5] == trajTrain.loc[nextIndex,5]):
    nextPointTime = trajTrain.loc[nextIndex,9] - trajTrain.loc[minIndex,10]
  else:
    nextPointTime = -1

  if (preIndex in trajTrain.index) & \
     (trajTrain.loc[minIndex,5] == trajTrain.loc[preIndex,5]):
    prePointTime = trajTrain.loc[minIndex,9] - trajTrain.loc[preIndex,10]
  else:
    prePointTime = -1

  return (nextPointTime,prePointTime)

# 2020/06/23

def getFullTime2(cube,trajTrainOri,mappingTrain):
    targetTrainOri = trajTrainOri.loc[(trajTrainOri[5] == cube[5]),]
    indexOri = mappingTrain.loc[(mappingTrain[2] == cube[5]) & (mappingTrain[4] == cube[1]) , 3]
    result = targetTrainOri.iloc[indexOri.to_list()[-1],]
    return result['postTime']

def getTestResult4OneTarget2(targetTrajNo,
                             trajTest,
                             trajTestOri,
                             mappingTest,
                             trajTrain,
                             trajTrainOri,
                             mappingTrain,
                             epLng, epLat):
  targetTraj = trajTest.loc[trajTest[5] == targetTrajNo,].copy()
  targetTrajOri = trajTestOri.loc[trajTestOri[5] == targetTrajNo,].copy()
  targetMapping = mappingTest.loc[mappingTest[2] == targetTrajNo,].copy()
  #result = ignoreBeginOrLastOnePointCube(targetTraj, targetTrajOri, targetMapping, epLng, epLat)
  #result = ignoreBeginOrLastOnePointCube_Origi(targetTraj, targetTrajOri, targetMapping)
  #if result is None:
  #  return (0, 0, 0, None)
  #targetNew, targetOriNew = result
  targetNew = targetTraj.copy()
  targetOriNew = targetTrajOri.copy()
  targetNew2 = targetNew.copy()
  realTime = targetNew.iloc[-1, 10] - targetNew.iloc[0, 9]
  realTime2 = realTime
  totalTime = 0
  n = targetNew.shape[0]
  commonTime = 0

  ratioOfMostGap = 0.7
  timeDiff = targetTrajOri[2].iloc[1:].to_numpy() - targetTrajOri[2].iloc[:-1].to_numpy()
  timeDiff = timeDiff.astype('int')
  timeDiff = numpy.sort(timeDiff)
  iarr = math.ceil(timeDiff.size * ratioOfMostGap)
  timeDiffTop = timeDiff[iarr] * 2

  tempExpResult = numpy.zeros((n, 4), dtype='int')
  tempExpResult[:, 0] = targetNew.iloc[:, 10] - targetNew.iloc[:, 9]
  listArea = []
  nextPointTime = 0
  prePointTime = 0
  preCubeTimeDiff = 0
  preNextPointTime = 0
  for i2 in range(n):
    # print(i2)

    if i2 > 0:
      targetNew.iloc[i2, 9] = targetNew.iloc[i2 - 1, 9] + commonTime
    area = getOverlayOneCubeWithPointAmount(targetNew.iloc[i2,],
                                            trajTrain,
                                            3600,
                                            epLng,
                                            epLat,
                                            10, 0.5)
    if area is None:
      print('ERROR: index=', i2, ' area is None')
      #preNextPointTime = -1
      continue
    commonTime,minIndex,dtwResult = getTimeByMinDtw(targetNew.iloc[i2, 1],
                                 area,
                                 targetOriNew,
                                 targetMapping,
                                 trajTrain,
                                 trajTrainOri,
                                 mappingTrain)
    commonTime2 = getFullTime2(trajTrain.loc[minIndex,],trajTrainOri,mappingTrain)
    commonTime += commonTime2
    '''
    nextPointTime, prePointTime = getTrainNext(trajTrain, minIndex)
    if (preNextPointTime == -1) & (prePointTime > -1) & (preCubeTimeDiff < timeDiffTop):
      tempExpResult[i2-1, 2] += prePointTime
      totalTime += prePointTime
      targetNew.iloc[i2, 9] += prePointTime
    tempExpResult[i2, 1] = commonTime
    if i2 < (n - 1):
      cubeTimeDiff = targetNew2.iloc[i2 + 1, 9] - targetNew2.iloc[i2, 10]
      preCubeTimeDiff = cubeTimeDiff
      if cubeTimeDiff > timeDiffTop:
        realTime2 -= cubeTimeDiff
        nextPointTime = 0
      elif nextPointTime > -1:
        commonTime += nextPointTime
    preNextPointTime = nextPointTime
    '''
    tempExpResult[i2, 2] = commonTime
    totalTime += commonTime
    tempExpResult[i2, 3] = dtwResult
  return (totalTime, realTime, realTime2, tempExpResult)



In [10]:
targetTrajNo = 14
totalTime,realTime,realTime2,tempExpResult = \
    getTestResult4OneTarget2(targetTrajNo,
                            trajTest,
                            trajTestOri,
                            mappingTest,
                            trajTrain,
                            trajTrainOri,
                            mappingTrain,
                            epLng,epLat)

In [11]:
print(totalTime,"\n",realTime,"\n",realTime2,"\n",tempExpResult)

2183.0 
 2082 
 2082 
 [[ 140    0  431  764]
 [ 198    0   40  444]
 [  50    0 1510  421]
 [1080    0  101  531]
 [1080    0  101  526]]


In [12]:
trajNoList = numpy.unique(trajTest[5])

expResult = numpy.zeros((trajNoList.max()+1,5),dtype='int')
#expResult[:,2] = 0
expResult[:,4] = list(range(trajNoList.max()+1))
#
#expTotalResult = 0

In [19]:
#
for targetTrajNo in trajNoList[100:200]:
    print(targetTrajNo, end=' ')
    totalTime,realTime,realTime2,tempExpResult = \
        getTestResult4OneTarget2(targetTrajNo,
                            trajTest,
                            trajTestOri,
                            mappingTest,
                            trajTrain,
                            trajTrainOri,
                            mappingTrain,
                            epLng,epLat)
    expResult[targetTrajNo,0] = 1
    expResult[targetTrajNo,1] = totalTime
    expResult[targetTrajNo,3] = realTime

101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 ERROR: index= 4  area is None
163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 

In [39]:
trajTest.loc[trajTest[5]==14,]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
97,6,0,2222,104.086630,30.642662,14,-1269,-314,-166,2152,2292,104.093264,104.079995,30.645447,30.642619
98,3,1,2441,104.076076,30.644613,14,-297,-21,-176,2342,2540,104.077630,104.074522,30.644766,30.644578
99,2,2,2756,104.073299,30.640135,14,4,-409,-89,2731,2781,104.073279,104.073319,30.641975,30.638295
100,2,3,3694,104.073052,30.637443,14,-53,-46,-139,3154,4234,104.073052,104.072497,30.637443,30.637026
101,3,4,4269,104.069523,30.638068,14,-569,232,158,4234,4304,104.072497,104.066549,30.637026,30.639110


In [17]:

with open('expResult1-.pickle','wb') as f:
	pickle.dump(expResult, f)

array([[   1, 1018,    0,  722,    1],
       [   1, 2703,    0, 2483,    2],
       [   1, 1251,    0, 1053,    3],
       [   1, 1567,    0, 1477,    4],
       [   1, 1933,    0, 1929,    5],
       [   1, 2461,    0, 1983,    6],
       [   1,  604,    0,  378,    7],
       [   1, 1037,    0, 1033,    8],
       [   1, 1161,    0, 1043,    9],
       [   1,  743,    0,  843,   10],
       [   1,  618,    0,  414,   11],
       [   1,  895,    0,  623,   12],
       [   1, 1037,    0, 2300,   13],
       [   1, 2183,    0, 2082,   14],
       [   1, 1440,    0, 1475,   15],
       [   1, 3653,    0, 2323,   16],
       [   1, 3316,    0, 1359,   17],
       [   1, 1544,    0, 1519,   18],
       [   1, 1426,    0, 1316,   19],
       [   1, 1515,    0, 1853,   20],
       [   1, 1868,    0, 2003,   21],
       [   1, 1376,    0, 1425,   22],
       [   1, 1976,    0, 1911,   23],
       [   1, 1414,    0, 1368,   24],
       [   1, 1214,    0, 1287,   25],
       [   1, 1386,    0,

In [25]:
expResult[800:810]

array([[   1,  555,    0, 1113,  800],
       [   0,    0,    0,    0,  801],
       [   0,    0,    0,    0,  802],
       [   0,    0,    0,    0,  803],
       [   0,    0,    0,    0,  804],
       [   0,    0,    0,    0,  805],
       [   0,    0,    0,    0,  806],
       [   0,    0,    0,    0,  807],
       [   0,    0,    0,    0,  808],
       [   0,    0,    0,    0,  809]])

In [57]:
expResult2 = expResult[numpy.where((expResult[:,0] == 1) & (expResult[:,1] >= 1000) )]
deviation = numpy.absolute(expResult2[:,1] - expResult2[:,3])
eachAPE = deviation * 100 / expResult2[:,3] 
eachAPE = eachAPE.astype(int)
#eachAPE[[49,6,11,60]] = 0
eachAPE

array([ 1,  5,  7, 12,  0, 63, 69,  1, 20, 11,  7,  3, 11, 13, 12, 42, 19,
        9,  4,  6, 19, 21, 11,  4, 39, 72, 20, 33, 24,  2,  6,  2,  4,  8,
       30,  9, 13,  8,  4,  4, 18,  2,  5])

In [58]:
eachAPE.mean()

15.651162790697674